####
## Playgroud data analysis

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [6]:
train.head()

,id,date,country,store,product,num_sold
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136950 entries, 0 to 136949
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        136950 non-null  int64 
 1   date      136950 non-null  object
 2   country   136950 non-null  object
 3   store     136950 non-null  object
 4   product   136950 non-null  object
 5   num_sold  136950 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 6.3+ MB


In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27375 entries, 0 to 27374
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       27375 non-null  int64 
 1   date     27375 non-null  object
 2   country  27375 non-null  object
 3   store    27375 non-null  object
 4   product  27375 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.0+ MB


In [9]:
full = train.append(test, ignore_index=True)

<ipython-input-9-6354882c2070>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full = train.append(test, ignore_index=True)


In [10]:
full.tail()

,id,date,country,store,product,num_sold
164320,164320,2022-12-31,Spain,Kagglazon,Using LLMs to Improve Your Coding,NaN
164321,164321,2022-12-31,Spain,Kagglazon,Using LLMs to Train More LLMs,NaN
164322,164322,2022-12-31,Spain,Kagglazon,Using LLMs to Win Friends and Influence People,NaN
164323,164323,2022-12-31,Spain,Kagglazon,Using LLMs to Win More Kaggle Competitions,NaN
164324,164324,2022-12-31,Spain,Kagglazon,Using LLMs to Write Better,NaN


In [11]:
full.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [12]:
full.date.value_counts()

2017-01-01    75
2021-01-03    75
2020-12-28    75
2020-12-29    75
2020-12-30    75
              ..
2018-12-26    75
2018-12-25    75
2018-12-24    75
2018-12-23    75
2022-12-31    75
Name: date, Length: 2191, dtype: int64

In [13]:
full.country.value_counts()

Argentina    32865
Canada       32865
Estonia      32865
Japan        32865
Spain        32865
Name: country, dtype: int64

In [14]:
full['product'].astype('category').value_counts()

Using LLMs to Improve Your Coding                 32865
Using LLMs to Train More LLMs                     32865
Using LLMs to Win Friends and Influence People    32865
Using LLMs to Win More Kaggle Competitions        32865
Using LLMs to Write Better                        32865
Name: product, dtype: int64

In [15]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
full['country'] = le.fit_transform(full['country'])
full['store'] = le.fit_transform(full['store'])
full['product'] = le.fit_transform(full['product'])

In [16]:
full.drop(['date'], axis=1, inplace=True)

In [17]:
train = full[full['num_sold'].notnull()]
test = full[full['num_sold'].isnull()]

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136950 entries, 0 to 136949
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        136950 non-null  int64  
 1   country   136950 non-null  int64  
 2   store     136950 non-null  int64  
 3   product   136950 non-null  int64  
 4   num_sold  136950 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 6.3 MB


In [19]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27375 entries, 136950 to 164324
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        27375 non-null  int64  
 1   country   27375 non-null  int64  
 2   store     27375 non-null  int64  
 3   product   27375 non-null  int64  
 4   num_sold  0 non-null      float64
dtypes: float64(1), int64(4)
memory usage: 1.3 MB


In [20]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [21]:
X = train.drop(['num_sold'], axis=1)
y = train['num_sold']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [22]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [23]:
score = xgb.score(X_test, y_test)
score

0.9844607320032566

In [24]:
X_test = test.drop(['num_sold'], axis=1)
y_pred = xgb.predict(X_test)

In [25]:
y_pred

array([ 45.40885 ,  50.41122 ,   6.871476, ..., 155.71202 , 644.1529  ,
       529.3971  ], dtype=float32)

In [26]:
# combine id and prediction

submit_sample = pd.read_csv('sample_submission.csv')
submit_sample.head()

,id,num_sold
0,136950,100
1,136951,100
2,136952,100
3,136953,100
4,136954,100


In [27]:
submit_sample['num_sold'] = y_pred

In [28]:
submit_sample.head()

,id,num_sold
0,136950,45.408852
1,136951,50.411221
2,136952,6.871476
3,136953,46.361713
4,136954,39.923256


In [29]:
submit_sample.to_csv('results/submit_xgb.csv', index=False)